In [1]:
from base_classification import *

In [2]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.datasets import make_classification
from sklearn.metrics import *


In [3]:
X, y = make_classification(2000)

In [4]:
from sklearn import datasets
iris = datasets.load_iris()

In [5]:
def eval_pipeline(additional_feats, X, y, verbose=True):
    #print(additional_feats)    
    pipeline = additional_feats[:]
    pipeline.append(('SVC', SVC()))
    model = Pipeline(pipeline[:])

    # split data into 10 folds
    kfold = KFold(n_splits=10, shuffle=False, random_state=42)
    results = cross_val_score(model, X, y, cv=kfold)
    if verbose:
        print("Result: {}".format(results.mean()))
    return results.mean()

In [6]:
# start...

In [7]:
bmars = BMARS(X)

In [8]:
interaction = 2
current_basis = bmars.export()['basis']
k = len(current_basis)+1
l = output_poisson_lambda(k)
bk, dk, ck = output_prob_state(k, l)

action = output_action(np.random.uniform(), bk, dk, ck)
basis = bmars.perform_action(action)
output = bmars_sample_basis(X, list(basis), {'signs':[-1, 1]})
proposed_model = BMARS(**bmars.export())
proposed_model.add_basis(**output)

alpha = acceptance_proba(X, y, l, interaction, 
                 bmars, 
                 proposed_model, mode=action)

print("Action: {}".format(action))
print("basis: {}".format(list(basis)))
print("output: {}".format(output))
print("alpha: {}".format(alpha))

Action: birth
basis: [1, 15]
output: {'sign': [-1, -1], 'basis': [1, 15], 'knot': array([ 0.10216263,  1.03593919])}
alpha: (1.0, {'proposal_ratio': 489148296.3330434, 'prior_ratio': 6.1331093709000817e-09, 'bayes_ratio': 0.99253798435673835})


In [19]:
def mh_iter(X, interaction, current_model, debug=True):
    current_basis = current_model.export()['basis']
    k = len(current_basis)+1
    l = output_poisson_lambda(k)
    bk, dk, ck = output_prob_state(k, l)

    action = output_action(np.random.uniform(), bk, dk, ck)
    basis = current_model.perform_action(action)
    output = bmars_sample_basis(X, list(basis), {'signs':[-1, 1]})
    proposed_model = BMARS(**current_model.export())
    proposed_model.add_basis(**output)

    alpha, accept_info = acceptance_proba(X, y, l, interaction, 
                     current_model, 
                     proposed_model, mode=action)

    if debug:
        print("Action: {}".format(action))
        print("basis: {}".format(list(basis)))
        print("output: {}".format(output))
        print("alpha: {}".format(alpha))
        print("bayes_ratio: {}".format(accept_info['bayes_ratio']))
        print("prior_ratio: {}".format(accept_info['prior_ratio']))
        print("proposal_ratio: {}".format(accept_info['proposal_ratio']))
    #
    return np.random.uniform() < alpha

In [20]:
mh_iter(X, 2, bmars)

Action: birth
basis: [19]
output: {'sign': [1], 'basis': [19], 'knot': array([-0.04620963])}
alpha: 1.0
bayes_ratio: 1.0032240730789896
prior_ratio: 2.863950910957339e-05
proposal_ratio: 104750.39877681342


True

In [21]:
mh_iter(X, 2, bmars, debug=False)

True